In [81]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy
import sqlite3
import re

In [2]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [2]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [4]:
engine = create_engine("sqlite:///final_project")

In [5]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [8]:
results = client.get("xwdj-i9he", limit=1160000)

In [9]:
results_df = pd.DataFrame.from_records(results)

In [10]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_lat,sr_location_lat_long,sr_status_date,sr_location_street_name,sr_created_date,sr_location_city,sr_location_y,sr_location_map_tile,sr_location_zip_code,sr_location_street_number
530602,"4709 CANDLETREE LN, AUSTIN, TX 78744",2,Phone,2018-10-24T13:38:09.000,3113741.99697997,Closed,ARR Missed Recycling,TRAVIS,675A,-97.74673768,...,30.18887143,"{'latitude': '30.18887143', 'needs_recoding': ...",2018-10-24T13:38:09.000,CANDLETREE,2018-08-20T14:01:46.000,AUSTIN,10041930.0014338,MJ15,78744,4709
425230,"300 E RIVERSIDE DR, AUSTIN, TX 78704",9,Spot311 Interface,2019-04-19T19:56:37.000,3114235.43491865,Closed,Dockless Mobility,TRAVIS,615E,-97.74341325,...,30.25461477,"{'latitude': '30.25461477', 'needs_recoding': ...",2019-04-19T19:56:37.000,RIVERSIDE,2019-04-19T18:17:58.000,AUSTIN,10065858.2728743,MJ21,78704,300
397523,"520 CARISMATIC LN, AUSTIN, TX 78748",5,Phone,2019-07-25T07:54:20.000,3093406.40151796,Closed,ARR Brush and Bulk,TRAVIS,703C,-97.81210427,...,30.15108153,"{'latitude': '30.15108153', 'needs_recoding': ...",2019-07-25T07:54:20.000,CARISMATIC,2019-06-07T13:04:14.000,AUSTIN,10027715.48211,MF12,78748,520
558122,"1032 PAYTON GIN RD, AUSTIN, TX 78758",4,Phone,2018-06-21T07:25:38.000,3124992.61997749,Closed,Austin Code - Request Code Officer,TRAVIS,526E,-97.706442,...,30.36089594,"{'latitude': '30.36089594', 'needs_recoding': ...",2018-06-21T07:25:38.000,PAYTON GIN,2018-06-21T07:15:23.000,AUSTIN,10104774.5495392,ML30,78758,1032
194225,"2223 SOUTH LAKESHORE BLVD, AUSTIN, TX 78741",3,Spot311 Interface,2020-08-22T14:19:26.000,3121805.66228940,Closed,Parking Violation Enforcement,TRAVIS,615Q,-97.7197383,...,30.24322857,"{'latitude': '30.24322857', 'human_address': '...",2020-08-22T14:19:26.000,SOUTH LAKESHORE,2020-08-22T13:24:01.000,AUSTIN,10061893.16955990,MK20,78741,2223
267940,"10614 SANS SOUCI PL, AUSTIN, TX 78759",10,Phone,2020-03-04T07:32:01.000,3105783.50844373,Closed,Loose Dog,TRAVIS,464U,-97.76598872,...,30.41260565,"{'latitude': '30.41260565', 'human_address': '...",2020-03-04T07:32:01.000,SANS SOUCI,2020-03-03T12:05:08.000,AUSTIN,10123137.34151120,MH34,78759,10614
995510,"6615 CRUZ ST, AUSTIN, TX",3,Phone,2015-04-02T15:58:57.000,3128819.73528324,Closed,Animal Control - Assistance Request,TRAVIS,616X,-97.69794015,...,30.22781939,"{'latitude': '30.22781939', 'needs_recoding': ...",2015-04-02T15:58:57.000,CRUZ,2015-04-02T10:11:50.000,AUSTIN,10056452.9340832,ML19,78741,6615
1027464,"8509 FM 969 RD, AUSTIN, TX",1,Phone,2014-12-16T13:56:08.000,3147469.48692161,Closed,School Zone - New/Review/Changes,TRAVIS,587Q,-97.63750223,...,30.27688094,"{'latitude': '30.27688094', 'needs_recoding': ...",2014-12-16T13:56:08.000,969,2014-12-16T11:24:35.000,AUSTIN,10074748.2786151,MP23,78725,8509
88773,"1112 W PARMER LN, AUSTIN, TX 78753",7,Mobile Device,2021-04-07T11:57:40.000,3133159.18178302,Closed,Austin Code - Request Code Officer,TRAVIS,466Y,-97.67924309,...,30.40805774,"{'latitude': '30.40805774', 'human_address': '...",2021-04-07T11:57:40.000,PARMER,2021-04-07T11:47:52.000,AUSTIN,10122125.46279420,MM34,78753,1112
416556,"8312 N IH 35 SB 23, AUSTIN, TX 78753",4,Web,2019-05-05T17:58:18.000,3128362.98099741,Closed,Austin Code - Request Code Officer,TRAVIS,526P,-97.69616604,...,30.34577373,"{'latitude': '30.34577373', 'needs_recoding': ...",2019-05-05T17:58:18.000,IH 35,2019-05-05T17:39:13.000,AUSTIN,10099352.9876463,ML29,78753,8312


In [11]:
len(results_df)

1151435

In [10]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [11]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [12]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [13]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [14]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [15]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [16]:
median = results_df['Total Seconds'].median(axis=0)

In [26]:
#median

69911.5

In [17]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [20]:
#print(results_df['Total Seconds'].max())

109367883.0


In [21]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_map_tile,sr_location_zip_code,sr_location_street_number,Created Date,Closed Date,Request Year,Open Time,Open Time delta,Total Seconds,Response Time
610168,"1520 RUTHERFORD LN, AUSTIN, TX 78754",1,Phone,2021-08-05T14:23:43.000,3133220.45539295,Closed,Animal Control - Assistance Request,TRAVIS,526Y,-97.68097996,...,MM28,78754,1520,2018-02-16 18:25:40,2021-08-05 14:23:43,2018,1265 days 19:58:03,1265 days 19:58:03,109367883.0,Slow Response


In [22]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

31898303.480000004

In [18]:
response = results_df['Response Time'].value_counts()
response

Quick Response    163163
Slow Response     163158
Name: Response Time, dtype: int64

In [19]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [20]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [21]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [32]:
#top_dept = results_df['Department'].value_counts()
#top_dept

Austin Code                 75872
Animal Services             60761
Austin Transportation       56381
Public Works                56162
Austin Resource Recovery    50856
Parks and Recreation         8728
Public Health                7979
Watershed Check              4887
Austin Water                 4058
Housing and Planning           87
Austin Fire                    65
Name: Department, dtype: int64

In [23]:
print(results_df.columns)

Index(['sr_location', 'sr_location_council_district',
       'sr_method_received_desc', 'sr_closed_date', 'sr_location_x',
       'sr_status_desc', 'sr_type_desc', 'sr_location_county',
       'sr_location_map_page', 'sr_location_long', 'sr_updated_date',
       'sr_number', 'sr_location_lat', 'sr_location_lat_long',
       'sr_status_date', 'sr_location_street_name', 'sr_created_date',
       'sr_location_street_number', 'sr_location_city', 'sr_location_y',
       'sr_location_map_tile', 'sr_location_zip_code', 'Created Date',
       'Closed Date', 'Request Year', 'Open Time', 'Open Time delta',
       'Total Seconds', 'Response Time', 'Department'],
      dtype='object')


In [118]:
dashboard_df.head()

,Address,Council_District,Report_Method,Complaint_Type,Longitude,Latitude,Created_Date,ZipCode,Open_Time,Department
297216,"6510 BERKMAN DR, AUSTIN, TX 78723",1,Phone,Loud Commercial Music,-97.68991552,30.31853425,2019-12-31T23:57:21.000,78723,1 days 18:16:30,Austin Code
297217,"1800 E STASSNEY LN, AUSTIN, TX 78744",2,Spot311 Interface,Loud Commercial Music,-97.76081753,30.19754356,2019-12-31T23:44:51.000,78744,1 days 18:28:54,Austin Code
297218,"400 E 7TH ST, AUSTIN, TX 78701",9,Web,Street Light Issue- Address,-97.7389242,30.26805529,2019-12-31T23:35:12.000,78701,1 days 07:28:47,Public Works
297219,"200 E 15TH ST, AUSTIN, TX 78701",1,Spot311 Interface,ARR Dead Animal Collection,-97.737741,30.27669722,2019-12-31T23:17:46.000,78701,1 days 09:48:16,Austin Resource Recovery
297220,"200 SAN JACINTO BLVD, AUSTIN, TX 78701",9,Phone,Loud Commercial Music,-97.74186188,30.26405608,2019-12-31T23:03:52.000,78701,1 days 19:09:52,Austin Code


In [22]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [93]:
dashboard_df = results_df.drop(columns=['sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Request Year', 'Open Time delta', 'Total Seconds', 'Response Time'])

In [95]:
dashboard_df = dashboard_df.drop(columns=['sr_updated_date', 'sr_number', 'sr_location_lat_long', 'sr_status_date','sr_location_street_name', 'sr_location_street_number', 'sr_location_city', 'sr_location_y'])

In [97]:
dashboard_df = dashboard_df.rename(columns={'sr_location':'Address', 'sr_location_council_district':'Council District', 'sr_method_received_desc':'Report Method', 'sr_type_desc':'Complaint Type', 'sr_location_long':'Longitude', 'sr_location_late':'Latitude', 'sr_created_date':'Created Date', 'sr_location_zip_code':'ZipCode'})

In [115]:
dashboard_df = dashboard_df.rename(columns={'Council District':'Council_District', 'Report Method':'Report_Method', 'Complaint Type':'Complaint_Type', 'Created Date':'Created_Date'})

In [117]:
dashboard_df = dashboard_df.rename(columns={'Open Time':'Open_Time'})

In [120]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


dashboard_df.to_sql('dashboard', con=engine, if_exists='replace')

engine.execute("SELECT * FROM dashboard").fetchall()

[(297216, '6510 BERKMAN DR, AUSTIN, TX 78723', '1', 'Phone', 'Loud Commercial Music', '-97.68991552', '30.31853425', '2019-12-31T23:57:21.000', '78723', 152190000000000, 'Austin Code'),
 (297217, '1800 E STASSNEY LN, AUSTIN, TX 78744', '2', 'Spot311 Interface', 'Loud Commercial Music', '-97.76081753', '30.19754356', '2019-12-31T23:44:51.000', '78744', 152934000000000, 'Austin Code'),
 (297218, '400 E 7TH ST, AUSTIN, TX 78701', '9', 'Web', 'Street Light Issue- Address', '-97.7389242', '30.26805529', '2019-12-31T23:35:12.000', '78701', 113327000000000, 'Public Works'),
 (297219, '200 E 15TH ST, AUSTIN, TX 78701', '1', 'Spot311 Interface', 'ARR Dead Animal Collection', '-97.737741', '30.27669722', '2019-12-31T23:17:46.000', '78701', 121696000000000, 'Austin Resource Recovery'),
 (297220, '200 SAN JACINTO BLVD, AUSTIN, TX 78701', '9', 'Phone', 'Loud Commercial Music', '-97.74186188', '30.26405608', '2019-12-31T23:03:52.000', '78701', 155392000000000, 'Austin Code'),
 (297221, '5602 LIBYAN 

In [42]:
mfi_data = "circle/MFI by Zipcode.csv"
pop_data = "circle/Population by Zipcode.csv"

mfi_df = pd.read_csv(mfi_data)
pop_df = pd.read_csv(pop_data)

In [70]:
mfi_df.head()

,Zip Code,Median Family Income
1,78610,88053
2,78613,115556
3,78617,63405
4,78641,108589
5,78652,103846


In [56]:
pop_df.head()

,GEO_ID,NAME,B01003_001E,B01003_001M
0,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total
1,8600000US78610,78610,38525,1382
2,8600000US78613,78613,81943,1254
3,8600000US78617,78617,28930,1917
4,8600000US78641,78641,72078,1773


In [61]:
#drop 0 index row
pop_df = pop_df.drop(0)

In [52]:
mfi_df.drop(0).head()

,GEO_ID,NAME,S1903_C03_015E
1,8600000US78610,78610,88053
2,8600000US78613,78613,115556
3,8600000US78617,78617,63405
4,8600000US78641,78641,108589
5,8600000US78652,78652,103846


In [45]:
#regex to match just zip code 
pop_df['NAME'] = pop_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [50]:
mfi_df['NAME'] = mfi_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [74]:
pop_df = pop_df.rename(columns={'Zip Code':'ZipCode','Total Population':'TotalPopulation'})

In [75]:
pop_df.head()

,ZipCode,TotalPopulation
1,78610,38525
2,78613,81943
3,78617,28930
4,78641,72078
5,78652,5374


In [63]:
pop_df = pop_df.drop(columns=['GEO_ID','B01003_001M'])

In [65]:
mfi_df = mfi_df.drop(0)

In [73]:
mfi_df = mfi_df.rename(columns={'Zip Code':'ZipCode','Median Family Income':'MedianFamilyIncome'})

In [69]:
mfi_df = mfi_df.drop(columns=['GEO_ID'])

In [76]:
mfi_df.head()

,ZipCode,MedianFamilyIncome
1,78610,88053
2,78613,115556
3,78617,63405
4,78641,108589
5,78652,103846


In [77]:
#read dfs into sqlite tables
pop_df.to_sql('population', con=engine, if_exists='replace')

engine.execute("SELECT * FROM population").fetchall()

[(1, '78610', '38525'),
 (2, '78613', '81943'),
 (3, '78617', '28930'),
 (4, '78641', '72078'),
 (5, '78652', '5374'),
 (6, '78653', '24313'),
 (7, '78660', '91300'),
 (8, '78664', '64686'),
 (9, '78681', '58155'),
 (10, '78701', '10848'),
 (11, '78702', '23671'),
 (12, '78703', '20407'),
 (13, '78704', '48746'),
 (14, '78705', '34549'),
 (15, '78712', '880'),
 (16, '78717', '30657'),
 (17, '78719', '1709'),
 (18, '78721', '12110'),
 (19, '78722', '7180'),
 (20, '78723', '35725'),
 (21, '78724', '25723'),
 (22, '78725', '8469'),
 (23, '78726', '14040'),
 (24, '78727', '30561'),
 (25, '78728', '21125'),
 (26, '78729', '28644'),
 (27, '78730', '9803'),
 (28, '78731', '27298'),
 (29, '78732', '17780'),
 (30, '78733', '8927'),
 (31, '78734', '19452'),
 (32, '78735', '18122'),
 (33, '78736', '10163'),
 (34, '78737', '17173'),
 (35, '78738', '15986'),
 (36, '78739', '21040'),
 (37, '78741', '52114'),
 (38, '78742', '887'),
 (39, '78744', '50123'),
 (40, '78745', '61907'),
 (41, '78746', '286

In [78]:
mfi_df.to_sql('income', con=engine, if_exists='replace')

engine.execute("SELECT * FROM income").fetchall()

[(1, '78610', '88053'),
 (2, '78613', '115556'),
 (3, '78617', '63405'),
 (4, '78641', '108589'),
 (5, '78652', '103846'),
 (6, '78653', '76372'),
 (7, '78660', '99733'),
 (8, '78664', '86640'),
 (9, '78681', '119784'),
 (10, '78701', '175993'),
 (11, '78702', '61016'),
 (12, '78703', '202837'),
 (13, '78704', '118264'),
 (14, '78705', '87870'),
 (15, '78712', '-'),
 (16, '78717', '129229'),
 (17, '78719', '77500'),
 (18, '78721', '44201'),
 (19, '78722', '104609'),
 (20, '78723', '73162'),
 (21, '78724', '49254'),
 (22, '78725', '58149'),
 (23, '78726', '132276'),
 (24, '78727', '96500'),
 (25, '78728', '78946'),
 (26, '78729', '100577'),
 (27, '78730', '204427'),
 (28, '78731', '146290'),
 (29, '78732', '198264'),
 (30, '78733', '200435'),
 (31, '78734', '138081'),
 (32, '78735', '146212'),
 (33, '78736', '115801'),
 (34, '78737', '156202'),
 (35, '78738', '157926'),
 (36, '78739', '173125'),
 (37, '78741', '47135'),
 (38, '78742', '54861'),
 (39, '78744', '52730'),
 (40, '78745', '7

In [110]:
  
# Query for INNER JOIN
sql = '''SELECT population.ZipCode, population.TotalPopulation, income.MedianFamilyIncome 
FROM population
INNER JOIN income
ON population.ZipCode = income.ZipCode;'''
  
# Executing the query
engine.execute(sql).fetchall()

[('78610', '38525', '88053'),
 ('78613', '81943', '115556'),
 ('78617', '28930', '63405'),
 ('78641', '72078', '108589'),
 ('78652', '5374', '103846'),
 ('78653', '24313', '76372'),
 ('78660', '91300', '99733'),
 ('78664', '64686', '86640'),
 ('78681', '58155', '119784'),
 ('78701', '10848', '175993'),
 ('78702', '23671', '61016'),
 ('78703', '20407', '202837'),
 ('78704', '48746', '118264'),
 ('78705', '34549', '87870'),
 ('78712', '880', '-'),
 ('78717', '30657', '129229'),
 ('78719', '1709', '77500'),
 ('78721', '12110', '44201'),
 ('78722', '7180', '104609'),
 ('78723', '35725', '73162'),
 ('78724', '25723', '49254'),
 ('78725', '8469', '58149'),
 ('78726', '14040', '132276'),
 ('78727', '30561', '96500'),
 ('78728', '21125', '78946'),
 ('78729', '28644', '100577'),
 ('78730', '9803', '204427'),
 ('78731', '27298', '146290'),
 ('78732', '17780', '198264'),
 ('78733', '8927', '200435'),
 ('78734', '19452', '138081'),
 ('78735', '18122', '146212'),
 ('78736', '10163', '115801'),
 ('7

In [112]:
sql = '''SELECT * FROM income;'''
engine.execute(sql).fetchall()

[(1, '78610', '88053'),
 (2, '78613', '115556'),
 (3, '78617', '63405'),
 (4, '78641', '108589'),
 (5, '78652', '103846'),
 (6, '78653', '76372'),
 (7, '78660', '99733'),
 (8, '78664', '86640'),
 (9, '78681', '119784'),
 (10, '78701', '175993'),
 (11, '78702', '61016'),
 (12, '78703', '202837'),
 (13, '78704', '118264'),
 (14, '78705', '87870'),
 (15, '78712', '-'),
 (16, '78717', '129229'),
 (17, '78719', '77500'),
 (18, '78721', '44201'),
 (19, '78722', '104609'),
 (20, '78723', '73162'),
 (21, '78724', '49254'),
 (22, '78725', '58149'),
 (23, '78726', '132276'),
 (24, '78727', '96500'),
 (25, '78728', '78946'),
 (26, '78729', '100577'),
 (27, '78730', '204427'),
 (28, '78731', '146290'),
 (29, '78732', '198264'),
 (30, '78733', '200435'),
 (31, '78734', '138081'),
 (32, '78735', '146212'),
 (33, '78736', '115801'),
 (34, '78737', '156202'),
 (35, '78738', '157926'),
 (36, '78739', '173125'),
 (37, '78741', '47135'),
 (38, '78742', '54861'),
 (39, '78744', '52730'),
 (40, '78745', '7

In [103]:
print(dashboard_df.columns)

Index(['Address', 'Council District', 'Report Method', 'Complaint Type',
       'Longitude', 'Latitude', 'Created Date', 'ZipCode', 'Open Time',
       'Department'],
      dtype='object')


In [121]:
# Join Census-level data to call data
sql = '''SELECT population.TotalPopulation, income.MedianFamilyIncome, dashboard.ZipCode, dashboard.Address, dashboard.Council_District, dashboard.Report_Method, dashboard.Complaint_Type, dashboard.Longitude, dashboard.Latitude, dashboard.Created_Date, dashboard.Open_Time, dashboard.Department 
FROM dashboard 
INNER JOIN population
ON dashboard.ZipCode = population.ZipCode
INNER JOIN income
ON dashboard.ZipCode = income.ZipCode;'''
  
# Executing the query
engine.execute(sql).fetchall()

[('35725', '73162', '78723', '6510 BERKMAN DR, AUSTIN, TX 78723', '1', 'Phone', 'Loud Commercial Music', '-97.68991552', '30.31853425', '2019-12-31T23:57:21.000', 152190000000000, 'Austin Code'),
 ('50123', '52730', '78744', '1800 E STASSNEY LN, AUSTIN, TX 78744', '2', 'Spot311 Interface', 'Loud Commercial Music', '-97.76081753', '30.19754356', '2019-12-31T23:44:51.000', 152934000000000, 'Austin Code'),
 ('10848', '175993', '78701', '400 E 7TH ST, AUSTIN, TX 78701', '9', 'Web', 'Street Light Issue- Address', '-97.7389242', '30.26805529', '2019-12-31T23:35:12.000', 113327000000000, 'Public Works'),
 ('10848', '175993', '78701', '200 E 15TH ST, AUSTIN, TX 78701', '1', 'Spot311 Interface', 'ARR Dead Animal Collection', '-97.737741', '30.27669722', '2019-12-31T23:17:46.000', 121696000000000, 'Austin Resource Recovery'),
 ('10848', '175993', '78701', '200 SAN JACINTO BLVD, AUSTIN, TX 78701', '9', 'Phone', 'Loud Commercial Music', '-97.74186188', '30.26405608', '2019-12-31T23:03:52.000', 155